In [2]:
import os.path
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

from utils.plots import plot
from utils.constants import Events, datetimes

/home/juand/Documents/vscode/DavidSierra/NMDB/.venv/lib/python3.12/site-packages/nolds/datasets.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
EVENT: Events = "Forbush Decrease"
event_replace = EVENT.replace(" ", "")
series_figure_path = "./figures/{event}/{date}/series"

In [4]:
paths = {
    date: list(Path(f"./data/ForbushDecrease/{date}").glob("*.csv"))
    for date in datetimes
}

In [5]:
for date, path in paths.items():
    for file in path:
        # Extract metadata from filename
        splits = file.stem.split("-")
        station_name = splits[0].split("_", 1)[0].upper()
        window_size = int(splits[1].split("_", 1)[1])
        ewm_alpha = splits[2].split("_")[2] if len(splits) == 3 else None
        bounds = pd.to_datetime(datetimes[date]["stations"][station_name])

        # Load data
        df = pd.read_csv(file, parse_dates=["datetime"])

        # Plot data
        fig, ax = plt.subplots(figsize=(14, 10))

        # Time Series data
        plot(df, ax=ax, freq_hours=1, are_metrics=False)

        # Vertical lines for event bounds
        ax.axvline(bounds[0], color="black", linestyle="--")
        ax.axvline(bounds[1], color="black", linestyle="--")

        # Title
        if ewm_alpha:
            title = rf"{station_name} Station | {EVENT} | EWM $\alpha={ewm_alpha}$"
            ewm_alpha_suffix = f"ewm_alpha_{ewm_alpha}"
        else:
            title = f"{station_name} Station | {EVENT}"
            ewm_alpha_suffix = ""

        ax.set(title=title)
        save_path = series_figure_path.format(event=event_replace, date=date)
        if not os.path.exists(save_path):
            os.makedirs(save_path, exist_ok=True)

        plt.savefig(
            f"{save_path}/{station_name}{f'-{ewm_alpha_suffix}' if ewm_alpha_suffix else ''}.pdf"
        )
        plt.close()